In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import math
import multiprocessing as mp

In [2]:
days_left = 365*1
# days_left = 10
today='2021-09-02'
yago = '2020-09-02'

In [3]:
def get_securities(date):
    page_size = 100
    all_rows = []
    start = 0
    url_temp = "https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities.json?date={date}&start={start}&iss.json=extended"
    r = requests.get(url_temp.format(start=start, date=date))
    res = r.json()
    all_rows += res[1]['history'][1]
    total = res[1]['history.cursor'][1][0]['TOTAL']
    pages = math.ceil(total/page_size)+1
    print("Date:{} Total:{}, Pages:{}".format(date, total, pages))
    if total <=100:
        return all_rows
    for page in range(1, pages):
        start = page*page_size
        r = requests.get(url_temp.format(start=start, date=date))
        res = r.json()
        all_rows += res[1]['history'][1]
    return all_rows

# res = get_securities("2020-09-01")



In [ ]:
secs = []
td = datetime.today()- timedelta(1)
dates = []
for i in range(days_left):
    dt = td - timedelta(i)
    dates.append(dt.strftime('%Y-%m-%d'))
    

In [ ]:
# pool = mp.Pool(mp.cpu_count())
# results = pool.map(get_securities, [date for date in dates])
# pool.close()

In [ ]:
secs = []
for r in results:
    secs += r

In [4]:
df_today = pd.DataFrame(get_securities(today))
df_1yago = pd.DataFrame(get_securities(yago))

df = df_today.append(df_1yago)
#df.to_excel("moex_bonds.xlsx", index=False)


Date:2021-09-02 Total:2378, Pages:25
Date:2020-09-02 Total:2059, Pages:22


In [13]:
bonds_prices = df[['SHORTNAME','BOARDID','TRADEDATE','SECID','LEGALCLOSEPRICE','MATDATE','DURATION','YIELDATWAP', 'COUPONPERCENT','FACEUNIT','FACEVALUE']]
bonds_prices

,SHORTNAME,BOARDID,TRADEDATE,SECID,LEGALCLOSEPRICE,MATDATE,DURATION,YIELDATWAP,COUPONPERCENT,FACEUNIT,FACEVALUE
0,UBANK11/22,TQOD,2021-09-02,AMUNIBB2AER2,103.20,2022-11-15,NaN,NaN,5.25,USD,100.0
1,UBANK02/24,TQOD,2021-09-02,AMUNIBB2DER6,103.60,2024-02-05,835.0,3.78,5.30,USD,100.0
2,МГор48-об,TQCB,2021-09-02,RU000A0JNYN1,100.00,2022-06-11,277.0,6.25,6.00,RUB,1000.0
3,НПФ 01,TQIR,2021-09-02,RU000A0JQ5C5,NaN,2029-05-06,NaN,NaN,5.00,RUB,1000.0
4,РЖД-19 обл,TQCB,2021-09-02,RU000A0JQ7Z2,102.00,2024-07-08,943.0,7.19,7.85,RUB,1000.0
...,...,...,...,...,...,...,...,...,...,...,...
2054,BCS06/25-9,TQCB,2020-09-02,XS2212035690,100.67,2025-06-30,NaN,NaN,9.50,RUB,100000.0
2055,BCS12/25-2,TQOD,2020-09-02,XS2213663797,100.30,2025-12-08,NaN,NaN,0.00,USD,1250.0
2056,BCS12/25,TQCB,2020-09-02,XS2213664092,99.42,2025-12-08,NaN,NaN,0.00,RUB,100000.0
2057,BCS GMf8,TQCB,2020-09-02,XS2215497830,100.70,2025-06-20,NaN,NaN,9.60,RUB,100000.0


In [14]:
#готовим списко облигаций
source_file = "data.xlsx" # исходник данных для получения universe
main_data = pd.read_excel(source_file, index_col=None, engine="openpyxl", parse_dates=True, sheet_name = "universe")

asset_class_list = ["ОФЗ", 'Облигации корпоративные', 'Облигации высокодоходные']
bonds_series = main_data.asset_class.isin(asset_class_list)
bonds = main_data[bonds_series]['code']
bonds_list = bonds.values.tolist()
bonds_list #получили список бондов

['RU000A102KG6',
 'RU000A102SV8',
 'RU000A103C46',
 'RU000A101QN1',
 'RU000A100Z91',
 'RU000A0JXQK2',
 'RU000A0ZYT40',
 'RU000A100VR4',
 'RU000A101Z74',
 'RU000A1002E8',
 'RU000A103G42',
 'RU000A103661',
 'RU000A1025U5',
 'RU000A103G75',
 'RU000A0ZYG52',
 'RU000A101QW2',
 'RU000A0ZYFC6',
 'RU000A0JUV08',
 'SU26229RMFS3',
 'SU26234RMFS3',
 'RU000A100K80',
 'SU26223RMFS6',
 'SU26222RMFS8',
 'SU26227RMFS7',
 'SU25084RMFS3',
 'SU26215RMFS2',
 'SU26226RMFS9',
 'SU26219RMFS4',
 'SU26207RMFS9',
 'SU26211RMFS1',
 'SU26220RMFS2',
 'SU26209RMFS5',
 'RU000A1005T9',
 'RU000A103JR3',
 'RU000A102KG6',
 'RU000A103M36',
 'RU000A0JWN63']

In [23]:
#фильтруем датасет с ценами по списку isin из нашего юниверса
filtered_bonds_prices = bonds_prices[bonds_prices.SECID.isin(bonds_list)]
#filtered_bonds_prices = filtered_bonds_prices[filtered_bonds_prices['BOARDID']=='TQCB'] #оставляем только котировки из стаканов
filtered_bonds_prices.sort_values(by=['SECID', 'TRADEDATE'])

,SHORTNAME,BOARDID,TRADEDATE,SECID,LEGALCLOSEPRICE,MATDATE,DURATION,YIELDATWAP,COUPONPERCENT,FACEUNIT,FACEVALUE
237,ГПБ БО-10,TQCB,2020-09-02,RU000A0JUV08,100.100,2024-09-26,24.0,NaN,6.89,RUB,1000.0
166,ГПБ БО-10,TQCB,2021-09-02,RU000A0JUV08,98.890,2024-09-26,373.0,7.23,6.00,RUB,1000.0
496,МОСТРЕСТ 7,TQCB,2020-09-02,RU000A0JWN63,103.390,2026-07-07,632.0,7.59,9.25,RUB,1000.0
326,МОСТРЕСТ 7,TQCB,2021-09-02,RU000A0JWN63,99.720,2026-07-07,305.0,9.62,9.25,RUB,1000.0
632,Роснфт1P4,TQCB,2020-09-02,RU000A0JXQK2,107.500,2027-04-22,865.0,5.63,8.65,RUB,1000.0
...,...,...,...,...,...,...,...,...,...,...,...
2088,ОФЗ 26227,TQOB,2021-09-02,SU26227RMFS7,101.710,2024-07-17,957.0,6.84,7.40,RUB,1000.0
1856,ОФЗ 26229,TQOB,2020-09-02,SU26229RMFS3,108.029,2025-11-12,1601.0,5.32,7.15,RUB,1000.0
2090,ОФЗ 26229,TQOB,2021-09-02,SU26229RMFS3,101.261,2025-11-12,1325.0,6.88,7.15,RUB,1000.0
1861,ОФЗ 26234,TQOB,2020-09-02,SU26234RMFS3,96.845,2025-07-16,1596.0,5.27,4.50,RUB,1000.0


In [61]:
#считаем историческую доходность бонда
#считаем изменение цены * на номинал и добавляем годовой купон

#Готовим данные по изменению вложений
pcntdf = filtered_bonds_prices.copy().reset_index()

groups_codes = list(set(pcntdf['SECID'].to_numpy().tolist()))

res = pd.DataFrame([])

for code in groups_codes:
    pcntdf_WITH_CODE = pcntdf[pcntdf['SECID']==code].sort_values('TRADEDATE')
    pcntdf_WITH_CODE['pct'] = pcntdf_WITH_CODE.LEGALCLOSEPRICE.pct_change(1).fillna(0)
    res = res.append(pcntdf_WITH_CODE)
res
#оставляем датарфейм на последню дату так как получили изменение цены
result = res[res['TRADEDATE']== today].copy()
result['1y_return'] = result['pct']*100+result['COUPONPERCENT']
res_1y = result[['SECID', '1y_return']].copy()
res_1y['field'] = '1y_return'
res_1y.columns = ['code','value','field']
res_1y = res_1y[['code','field','value']]

In [62]:
#для облигаций в качестве меры рика волатильности используем дюрацию в годах
dur = res[res['TRADEDATE']== today].copy()
dur['volatility'] = dur['DURATION']/365
dur['field'] = 'volatility'
vol = dur[['SECID','field', 'volatility']].copy()
vol.columns = ['code','field','value']

In [63]:
bonds_vol_ret_table = res_1y.append(vol)
bonds_vol_ret_table

,code,field,value
2,RU000A0JXQK2,1y_return,3.431395
9,RU000A100VR4,1y_return,2.186730
14,RU000A1025U5,1y_return,5.700000
30,SU26222RMFS8,1y_return,0.877674
11,RU000A101QN1,1y_return,1.094274
...,...,...,...
1,RU000A0JWN63,volatility,0.835616
6,RU000A1002E8,volatility,1.350685
28,SU26219RMFS4,volatility,4.150685
33,SU26227RMFS7,volatility,2.621918


In [64]:
#оцениваем потенциал доходности по облигациям по сценарию
#тащим сценарий
scenarios_data = pd.read_excel(source_file, index_col=None, engine="openpyxl", parse_dates=True, sheet_name = "scenario")
scenarios_dict = {}
scenarios_name_list = list(set(scenarios_data['scenario'].to_numpy().tolist()))
for scenario in scenarios_name_list:
    if scenario not in scenarios_dict:
        scenarios_dict[scenario] = {}
    sc = scenarios_data[scenarios_data['scenario']==scenario].to_dict('records')
    for row in sc:
        scenarios_dict[scenario][row['rate']] = row['forecast_value']
        
scenarios_dict

{'positive': {'RU_RATE': 5.0,
  '10Y_USD': 1.4,
  'RU_CPI': 4.0,
  'IMOEX': 5401.35,
  'USDRUB': 69.2,
  'SPX': 5986.754400000001,
  'RUCBITR': 501.91872},
 'base': {'RU_RATE': 5.5,
  '10Y_USD': 1.4,
  'RU_CPI': 4.5,
  'IMOEX': 4934.566666666667,
  'USDRUB': 70.2,
  'SPX': 5366.913666666667,
  'RUCBITR': 501.91872},
 'negative': {'RU_RATE': 6.0,
  '10Y_USD': 3.0,
  'RU_CPI': 9.0,
  'IMOEX': 3401.4449999999997,
  'USDRUB': 83.0,
  'SPX': 3855.1155000000003,
  'RUCBITR': 460.8}}

In [70]:
#тянем значение ставки последнее
rates_data = pd.read_excel(source_file, index_col=None, engine="openpyxl", parse_dates=True, sheet_name = "rates_history")

6.5

In [71]:
cur_rurate = rates_data[rates_data['code']=='RU_RATE'].iloc[-1]['price']
cur_rurate

6.5

In [ ]:
[['SECID', 'LEGALCLOSEPRICE', 'MATDATE', 'DURATION', 'COUPONPERCENT', 'FACEVALUE']]